<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment1_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [89]:
import numpy as np

In [ ]:
import tarfile
my_tar = tarfile.open('/content/lingspam_public.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()

In [90]:
import os
train_path = '/content/lingspam_public/lemm_stop/part1'  # for training      #spams: spmsg*.txt
test_path = '/content/lingspam_public/lemm_stop/part1'   # for testing

def to_dict (path):
  data_dict = dict()
  data_dict[1] = []
  data_dict[0] = []
  for file in os.listdir(path):  
    doc = open (path + '/'+ file, 'r')
    if 'spmsg' in file:
      data_dict[1].append(doc.read())
    else:
      data_dict[0].append(doc.read())
  print ('number of spams: {}'.format(len(data_dict[1])))
  print ('number of not_spams: {}'.format(len(data_dict[0])))
  n_docs = len(os.listdir(path))
  return data_dict, n_docs

# def to_bow (doc, class):
#   wbag[class] = []
#   for token in doc.split():
#     wbag.append(token)
#   return wbag

def count_words(data_dict):
  tf = {}
  tf[0], tf[1] = {}, {}
  all_docs = data_dict[0] + data_dict[1]
  print('herrrrrrrrrrrrrrreeeee', len(all_docs))
  temp0, temp1 = {},{}
  for doc in data_dict[0]:
    for token in doc.split():
      temp0[token] = temp0.get(token, 0)+1
    tf[0] = temp0
  for doc in data_dict[1]: 
    for token in doc.split():
      temp1[token] = temp1.get(token, 0)+1
    tf[1] = temp1
  return tf

def to_bow (data_dict):
  bows = {}
  bows[0], bows[1] = [], []
  for doc in data_dict[0]:
    bows[0].extend(doc.split())
  for doc in data_dict[1]:
    bows[1].extend(doc.split())
  return bows

def logprior(data_dict, n_docs):
  logprior = {}
  n_spams = len(data_dict[1])
  n_nonspams = len(data_dict[0])
  logprior[1] = np.log(n_docs/n_spams)
  logprior[0] = np.log(n_docs/n_nonspams)
  return logprior

# spam_bow = to_bow (training)[1]
# nonspam_bow = to_bow (training)[0]
# tf_spam = count_words(training)[1]
# tf_nonspam = count_words(training)[0]
# logprior_spam = logprior(training, n_docs_train)[1]
# logprior_notspam = logprior(training, n_docs_train)[0]

def train_NB (train_path):
  training, n_docs_train = to_dict (train_path)
  bows = to_bow (training)
  set_V = set(to_bow (training)[1] + to_bow (training)[0])
  tf = count_words(training)
  logpriors = logprior(training,n_docs_train)
  loglikelihood = {}
  loglikelihood[0], loglikelihood[1] = {}, {}
  for c in training.keys():
    # n_c = len(training[c])
    # logprior_c = np.log(n_c/n_docs_train)
    bow_c = bows[c]
    loglikelihood_c = {}
    for w in set_V:
      if w in tf[c]:
        count_w = tf[c][w]
        #print(count_w)
      else:
        count_w = 0
      loglikelihood_w = np.log((count_w + 1)/(len(bow_c)+1))
      loglikelihood_c[w] = loglikelihood_w
    loglikelihood[c] = loglikelihood_c
  return logpriors, loglikelihood, set_V

training_result = train_NB (train_path)


number of spams: 48
number of not_spams: 241
herrrrrrrrrrrrrrreeeee 289


In [97]:
testing, n_docs_test = to_dict (test_path)

X_test = testing[0]+ testing[1]
y_true = [0]*len(testing[0]) + [1]*len(testing[1])
def test_NB (test_path):
  logpriors= logprior(testing,n_docs_test)
  prediction = []
  X_test = testing[0]+ testing[1]
  for doc in X_test:
    spam_loglikelihoods, nonspam_loglikelihoods = [], []
    spam_score=0
    nonspam_score = 0
    for w in doc.split():
      if w not in training_result[2]: continue
      spam_loglikelihoods.append(training_result[1][1][w] if w in training_result[1][1].keys() else 0)
      nonspam_loglikelihoods.append(training_result[1][0][w] if w in training_result[1][0].keys() else 0)
    spam_score += sum(spam_loglikelihoods)
    nonspam_score += sum(nonspam_loglikelihoods)
    spam_score += logpriors [1]
    nonspam_score += logpriors [0]
    if spam_score > nonspam_score:
      prediction.append(1)
    else:
      prediction.append(0)
  return prediction

y_pred = test_NB (test_path)
print(y_true)
print(y_pred)

number of spams: 48
number of not_spams: 241
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [101]:
import sklearn
from sklearn.metrics import accuracy_score
print('accuracy on test set: {}'.format(sklearn.metrics.accuracy_score(y_true, y_pred)))

accuracy on test set: 0.9826989619377162


In [ ]:
# Testing


0.18162975462177713

In [ ]:
def fitNB (X, y):

def predict (test_set):
  for doc in test_set:

In [ ]:

class NaiveBayes:
    
    def __init__(self,unique_classes):
        
        self.classes=unique_classes # Constructor is sinply passed with unique number of classes of the training set
        

    def addToBow(self,example,dict_index):
        
        '''
            Parameters:
            1. example 
            2. dict_index - implies to which BoW category this example belongs to
            What the function does?
            -----------------------
            It simply splits the example on the basis of space as a tokenizer and adds every tokenized word to
            its corresponding dictionary/BoW
            Returns:
            ---------
            Nothing
        
       '''
        
        if isinstance(example,np.ndarray): example=example[0]
     
        for token_word in example.split(): #for every word in preprocessed example
          
            self.bow_dicts[dict_index][token_word]+=1 #increment in its count
            
    def train(self,dataset,labels):
        
        '''
            Parameters:
            1. dataset - shape = (m X d)
            2. labels - shape = (m,)
            What the function does?
            -----------------------
            This is the training function which will train the Naive Bayes Model i.e compute a BoW for each
            category/class. 
            Returns:
            ---------
            Nothing
        
        '''
    
        self.examples=dataset
        self.labels=labels
        self.bow_dicts=np.array([defaultdict(lambda:0) for index in range(self.classes.shape[0])])
        
        #only convert to numpy arrays if initially not passed as numpy arrays - else its a useless recomputation
        
        if not isinstance(self.examples,np.ndarray): self.examples=np.array(self.examples)
        if not isinstance(self.labels,np.ndarray): self.labels=np.array(self.labels)
            
        #constructing BoW for each category
        for cat_index,cat in enumerate(self.classes):
          
            all_cat_examples=self.examples[self.labels==cat] #filter all examples of category == cat
            
            #get examples preprocessed
            
            cleaned_examples=[preprocess_string(cat_example) for cat_example in all_cat_examples]
            
            cleaned_examples=pd.DataFrame(data=cleaned_examples)
            
            #now costruct BoW of this particular category
            np.apply_along_axis(self.addToBow,1,cleaned_examples,cat_index)
            
                
        ###################################################################################################
        
        '''
            Although we are done with the training of Naive Bayes Model BUT!!!!!!
            ------------------------------------------------------------------------------------
            Remember The Test Time Forumla ? : {for each word w [ count(w|c)+1 ] / [ count(c) + |V| + 1 ] } * p(c)
            ------------------------------------------------------------------------------------
            
            We are done with constructing of BoW for each category. But we need to precompute a few 
            other calculations at training time too:
            1. prior probability of each class - p(c)
            2. vocabulary |V| 
            3. denominator value of each class - [ count(c) + |V| + 1 ] 
            
            Reason for doing this precomputing calculations stuff ???
            ---------------------
            We can do all these 3 calculations at test time too BUT doing so means to re-compute these 
            again and again every time the test function will be called - this would significantly
            increase the computation time especially when we have a lot of test examples to classify!!!).  
            And moreover, it doensot make sense to repeatedly compute the same thing - 
            why do extra computations ???
            So we will precompute all of them & use them during test time to speed up predictions.
            
        '''
        
        ###################################################################################################
      
        prob_classes=np.empty(self.classes.shape[0])
        all_words=[]
        cat_word_counts=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
           
            #Calculating prior probability p(c) for each class
            prob_classes[cat_index]=np.sum(self.labels==cat)/float(self.labels.shape[0]) 
            
            #Calculating total counts of all the words of each class 
            count=list(self.bow_dicts[cat_index].values())
            cat_word_counts[cat_index]=np.sum(np.array(list(self.bow_dicts[cat_index].values())))+1 # |v| is remaining to be added
            
            #get all words of this category                                
            all_words+=self.bow_dicts[cat_index].keys()
                                                     
        
        #combine all words of every category & make them unique to get vocabulary -V- of entire training set
        
        self.vocab=np.unique(np.array(all_words))
        self.vocab_length=self.vocab.shape[0]
                                  
        #computing denominator value                                      
        denoms=np.array([cat_word_counts[cat_index]+self.vocab_length+1 for cat_index,cat in enumerate(self.classes)])                                                                          
      
        '''
            Now that we have everything precomputed as well, its better to organize everything in a tuple 
            rather than to have a separate list for every thing.
            
            Every element of self.cats_info has a tuple of values
            Each tuple has a dict at index 0, prior probability at index 1, denominator value at index 2
        '''
        
        self.cats_info=[(self.bow_dicts[cat_index],prob_classes[cat_index],denoms[cat_index]) for cat_index,cat in enumerate(self.classes)]                               
        self.cats_info=np.array(self.cats_info)                                 
                                              
                                              
    def getExampleProb(self,test_example):                                
        
        '''
            Parameters:
            -----------
            1. a single test example 
            What the function does?
            -----------------------
            Function that estimates posterior probability of the given test example
            Returns:
            ---------
            probability of test example in ALL CLASSES
        '''                                      
                                              
        likelihood_prob=np.zeros(self.classes.shape[0]) #to store probability w.r.t each class
        
        #finding probability w.r.t each class of the given test example
        for cat_index,cat in enumerate(self.classes): 
                             
            for test_token in test_example.split(): #split the test example and get p of each test word
                
                ####################################################################################
                                              
                #This loop computes : for each word w [ count(w|c)+1 ] / [ count(c) + |V| + 1 ]                               
                                              
                ####################################################################################                              
                
                #get total count of this test token from it's respective training dict to get numerator value                           
                test_token_counts=self.cats_info[cat_index][0].get(test_token,0)+1
                
                #now get likelihood of this test_token word                              
                test_token_prob=test_token_counts/float(self.cats_info[cat_index][2])                              
                
                #remember why taking log? To prevent underflow!
                likelihood_prob[cat_index]+=np.log(test_token_prob)
                                              
        # we have likelihood estimate of the given example against every class but we need posterior probility
        post_prob=np.empty(self.classes.shape[0])
        for cat_index,cat in enumerate(self.classes):
            post_prob[cat_index]=likelihood_prob[cat_index]+np.log(self.cats_info[cat_index][1])                                  
      
        return post_prob
    
   
    def test(self,test_set):
      
        '''
            Parameters:
            -----------
            1. A complete test set of shape (m,)
            
            What the function does?
            -----------------------
            Determines probability of each test example against all classes and predicts the label
            against which the class probability is maximum
            Returns:
            ---------
            Predictions of test examples - A single prediction against every test example
        '''       
       
        predictions=[] #to store prediction of each test example
        for example in test_set: 
                                              
            #preprocess the test example the same way we did for training set exampels                                  
            cleaned_example=preprocess_string(example) 
             
            #simply get the posterior probability of every example                                  
            post_prob=self.getExampleProb(cleaned_example) #get prob of this example for both classes
            
            #simply pick the max value and map against self.classes!
            predictions.append(self.classes[np.argmax(post_prob)])
                
        return np.array(predictions) 
view rawNB_class.py hosted with ❤ by GitHub

In [ ]:
sentence = 'ajccdkckc ojcdojdco oef9ff ojsosso fofekd ihijso ihis hisj hios'
bow_dicts = dict()
for token in sentence.split():
  bow_dicts['spam'][token]+=1
bow_dicts

KeyError: ignored